# Data Analysis experiment

this notebook is a personal project using the student scores from **UERJ** (university of the state of rio de janeiro) entrance exam to make some insights about the data and the overall performance on this test

In [1]:
# imports

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tabula
%matplotlib inline

import pandas as pd

# Set the maximum number of rows and columns to display
pd.set_option('display.max_rows', None)  # None means display all rows
pd.set_option('display.max_columns', None)  # None means display all columns

### 1. import data

data can be obtained from theu niversity official website:

https://www.vestibular.uerj.br/?page_id=14037

In [2]:
pdfs_path = 'data/'

# iterate over data directory and append all pdf's to a dataframe

dataframe = pd.DataFrame()

for pdf in os.listdir(pdfs_path):
    # combine all pdf's
    pdf_join = os.path.join(pdfs_path, pdf)

    pdf_data = tabula.read_pdf(pdf_join, pages='all')

    # append the data to the dataframe
    dataframe = dataframe.append(pdf_data, ignore_index=True)

Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'
Got stderr: dez 22, 2023 10:08:57 PM org.apache.pdfbox.rendering.PDFRenderer suggestKCMS
INFORMAÇÕES: Your current java version is: 1.8.0_51
dez 22, 2023 10:08:57 PM org.apache.pdfbox.rendering.PDFRenderer suggestKCMS
INFORMAÇÕES: To get higher rendering speed on old java 1.8 or 9 versions,
dez 22, 2023 10:08:57 PM org.apache.pdfbox.rendering.PDFRenderer suggestKCMS
INFORMAÇÕES:   update to the latest 1.8 or 9 version (>= 1.8.0_191 or >= 9.0.4),
dez 22, 2023 10:08:57 PM org.apache.pdfbox.rendering.PDFRenderer suggestKCMS
INFORMAÇÕES:   or
dez 22, 2023 10:08:57 PM org.apache.pdfbox.rendering.PDFRenderer suggestKCMS
INFORMAÇÕES:   use the option -Dsun.java2d.cmm=sun.java2d.cmm.kcms.KcmsServiceProvider
dez 22, 2023 10:08:57 PM org.apache.pdfbox.rendering.PDFRenderer suggestKCMS
INFORMAÇÕES:   or call System.setProperty("sun.java2d.cmm", "sun.java2d.cmm.kcms.KcmsServiceProvider")

C:\Users\USER\AppData\Loca

In [4]:
print(f'number of candidates: {len(dataframe)}')

number of candidates: 36045


### 2. data cleaning

In [5]:
modified_df = dataframe.copy()
modified_df[0:5]

,Inscrição,Nome do Candidato L.EST,Unnamed: 0,HIS,GEO,BIO,FIS,MAT,QUI,LPL,RED,Nome do Candidato,L.EST
0,243016253-0,ABDIEL FELIPE DE OLIVEIRA SILVA,NaN,NaN,NaN,NaN,NaN,"0,00",NaN,"0,00","1,00",NaN,NaN
1,243007702-3,ABEL MOSCHEN CARVALHO,NaN,NaN,NaN,"14,25",NaN,NaN,"13,75",NaN,"12,00",NaN,NaN
2,243030973-3,ABEL VIEIRA DE RESENDE,NaN,FALTA,NaN,NaN,NaN,NaN,NaN,FALTA,FALTA,NaN,NaN
3,243009357-0,ABIGAIL CRISTINA DA SILVA REZENDE,NaN,NaN,NaN,"7,75",NaN,NaN,"7,00",NaN,"8,00",NaN,NaN
4,243025508-0,ABIGAIL CRISTINA SALDANHA LEITE REZENDE DE MAT...,NaN,"6,50",NaN,NaN,NaN,NaN,NaN,NaN,"12,00",NaN,NaN


In [6]:
for index, row in modified_df.iterrows():
    if type(row['Nome do Candidato']) is float:
        row['Nome do Candidato'] = row['Nome do Candidato L.EST']



In [7]:
# remove columns 'Nome do Candidato L.EST' and 'Unnamed: 0'

columns_to_remove = ['Unnamed: 0', 'Nome do Candidato L.EST']

modified_df = modified_df.drop(columns=columns_to_remove)

In [9]:
# remove items with 'FALTA' (those were the students that weren't present at the day of the 


# Identify rows where 'RED' column is 'FALTA'
rows_to_remove = modified_df[modified_df['RED'] == 'FALTA'].index

# Use boolean indexing to remove the specified rows
modified_df = modified_df.drop(index=rows_to_remove)

rows_to_remove = modified_df[modified_df['RED'] == 'ELIM'].index

modified_df = modified_df.drop(index=rows_to_remove)

In [10]:
# change order of columns for mor organization
order = ['Inscrição', 'Nome do Candidato', 'HIS', 'GEO', 'BIO', 'FIS', 'MAT', 'QUI', 'LPL', 'L.EST', 'RED']

modified_df = modified_df[order]

### 3. data analysis

we want to discover some things:


- top 90% of the notes in math & physics and biology & chemistry
- who will be approved in medicine school
- mean ad variance of each displyne score



#### 3.1 top 90% of notes

#### top 90% notes in physics & math

In [11]:
# first we need to convert the scores to float numebers

df_names = modified_df[['Inscrição', 'Nome do Candidato']]
df_scores = modified_df[['HIS', 'GEO', 'BIO', 'FIS', 'MAT', 'QUI', 'LPL', 'L.EST', 'RED']]

In [12]:
def string_to_float(s):
    if type(s) is not str:
        return None
    return float(s.replace(',', '.'))

df_scores = df_scores.applymap(string_to_float)





In [13]:
fis_array = np.array(df_scores['FIS'].dropna())
mat_array = np.array(df_scores['MAT'].dropna())
bio_array = np.array(df_scores['BIO'].dropna())
qui_array = np.array(df_scores['QUI'].dropna())
red_array = np.array(df_scores['RED'].dropna())

In [14]:
print(f'physics 10% of  best students {np.percentile(fis_array, 90)}') # --> top 10% of sthe students got this grade
print(f'math 10% of  best students {np.percentile(mat_array, 90)}')
print(f'biology 10% of  best students {np.percentile(bio_array, 90)}')
print(f'chemestry 10% of  best students {np.percentile(qui_array, 90)}')

physics 10% of  best students 16.5
math 10% of  best students 15.5
biology 10% of  best students 11.25
chemestry 10% of  best students 12.0


In [18]:
# get grades that will probrabily be in med school

# Get the 57 highest numbers
k = 57
highest_numbers_bio = np.partition(bio_array, -k)[-k:]

print("57 highest numbers in biology:", highest_numbers_bio)


57 highest numbers in biology: [16.25 16.25 16.25 16.25 16.25 16.25 16.25 16.25 16.25 16.25 16.25 16.25
 16.5  16.25 16.25 16.5  16.5  16.5  16.5  16.5  16.5  16.5  16.5  16.75
 17.5  17.   17.25 16.75 18.   16.5  16.75 17.   17.   16.5  17.   16.75
 16.75 17.25 18.5  17.5  17.25 17.5  17.   16.75 16.75 18.   17.25 17.
 17.5  17.   16.5  17.25 17.   17.   17.   17.75 17.25]


In [19]:
# get grades that will probrabily be in med school

# Get the 57 highest numbers
k = 57
highest_numbers_qui = np.partition(qui_array, -k)[-k:]

print("57 highest numbers in chemestry:", highest_numbers_qui)

57 highest numbers in chemestry: [17.5  17.5  17.5  17.75 18.   17.75 17.75 18.   18.   18.   17.75 17.75
 18.   17.75 18.   17.75 17.75 18.   17.75 17.75 17.75 18.25 17.75 17.75
 18.   18.25 18.   18.25 18.   17.75 18.25 18.25 18.   18.25 18.   17.75
 17.75 18.   18.5  19.   19.   18.5  18.75 18.5  18.5  18.75 19.25 19.25
 18.5  19.   18.5  19.25 19.   19.   20.   18.5  19.  ]


In [17]:
# grades for math & physics

# Get the 66 highest numbers
k = 66
highest_numbers_mat = np.partition(mat_array, -k)[-k:]

print("66 highest numbers in math:", highest_numbers_mat)

# Get the 57 highest numbers
highest_numbers_fis = np.partition(fis_array, -k)[-k:]

print("66 highest numbers in physics:", highest_numbers_fis)

66 highest numbers in math: [19.   19.25 19.25 19.25 19.25 19.25 19.25 19.25 19.25 19.25 19.25 19.25
 19.25 19.25 19.25 19.5  19.5  19.25 19.5  19.5  19.25 19.25 19.5  19.25
 19.25 19.25 19.5  19.5  19.5  19.5  19.5  19.5  19.5  19.5  19.5  19.75
 19.25 19.25 19.5  19.75 19.75 20.   20.   20.   20.   20.   19.75 20.
 20.   20.   20.   20.   20.   20.   19.75 20.   20.   20.   20.   20.
 20.   20.   20.   20.   20.   20.  ]
66 highest numbers in physics: [19.5  19.5  19.5  19.5  19.5  19.5  19.5  19.5  19.5  19.5  19.5  19.5
 19.5  19.5  19.75 19.75 19.75 19.75 19.75 19.75 20.   20.   19.75 20.
 20.   20.   20.   20.   19.75 19.75 19.75 20.   19.75 19.75 19.75 20.
 20.   20.   19.75 20.   20.   20.   20.   20.   20.   20.   20.   20.
 20.   20.   20.   20.   20.   20.   20.   20.   20.   20.   20.   20.
 20.   20.   20.   20.   20.   20.  ]


In [21]:
cs_students_mat = np.random.choice(mat_array, size=int(len(mat_array)* 0.25))

k = 66
highest_numbers_cs = np.partition(cs_students_mat, -k)[-k:]

print("66 highest numbers in computer science:", highest_numbers_cs)


66 highest numbers in computer science: [17.5  17.5  17.5  17.5  18.75 18.   19.75 20.   20.   17.5  17.75 19.25
 17.75 19.25 18.25 17.75 18.   18.   17.75 18.25 17.5  19.25 18.   18.
 18.5  19.5  18.25 18.25 17.5  18.25 19.75 18.25 19.25 19.   20.   17.5
 18.   17.75 18.75 20.   17.75 17.75 19.5  20.   17.5  19.25 17.5  17.75
 18.5  19.5  17.75 18.5  19.   19.25 17.5  17.75 19.75 17.5  17.75 19.25
 18.25 19.   18.5  18.75 20.   17.75]


In [22]:
cs_students_fis = np.random.choice(fis_array, size=int(len(mat_array)* 0.25))

k = 66
highest_numbers_cs = np.partition(cs_students_fis, -k)[-k:]

print("66 highest numbers in computer science:", highest_numbers_cs)



66 highest numbers in computer science: [18.5  18.5  18.75 19.   18.75 18.5  18.75 18.75 18.75 18.75 19.   19.
 18.75 18.75 19.   18.5  19.   19.   19.   19.   19.   19.5  19.5  19.5
 19.5  19.25 20.   20.   19.5  19.   19.25 19.5  19.25 19.75 19.5  19.25
 19.5  19.5  19.   19.5  19.   19.5  20.   19.75 20.   19.75 20.   19.5
 19.5  20.   19.75 20.   20.   19.75 20.   19.75 19.75 19.75 20.   20.
 19.   20.   19.5  19.75 20.   19.75]


In [23]:
cs_students_red = np.random.choice(mat_array, size=int(len(red_array)/32))

k = 66
highest_numbers_cs = np.partition(cs_students_red, -k)[-k:]

print(len(red_array))
print(len(cs_students_red))
print("66 highest numbers in computer science:", highest_numbers_cs)

32173
1005
66 highest numbers in computer science: [17.   17.25 17.25 17.25 17.25 17.   17.5  17.5  17.5  17.5  17.5  17.5
 18.   18.   17.75 18.   17.75 17.75 18.   18.   18.25 18.   17.75 17.75
 17.75 18.   18.   17.75 18.25 18.   17.75 17.75 17.75 18.   17.5  17.5
 18.25 17.5  18.25 19.   20.   20.   20.   19.   19.5  18.5  20.   19.5
 18.75 18.5  19.75 18.75 20.   19.25 19.25 20.   19.25 19.5  18.25 19.5
 20.   19.5  18.75 19.   18.5  20.  ]


make a `DataFrame` with dicovered data

our `DataFrame` will have:
- data of biology, chemestry, math, essay and physics
- grades of top 10% of the students
- number of people doing these acceptance exams
- average grade
- variance of grade

In [29]:
# 10% best grades
top_10_bio = np.percentile(bio_array, 90)
top_10_qui = np.percentile(qui_array, 90)
top_10_mat = np.percentile(mat_array, 90)
top_10_fis = np.percentile(fis_array, 90)
top_10_red = np.percentile(red_array, 90)

In [31]:
# number of candidates
mat_candidates = len(mat_array)
fis_candidates = len(fis_array)
bio_candidats = len(bio_array)
qui_candidates = len(qui_array)
red_candidates = len(red_array)

In [32]:
# average grade
mat_avg = np.mean(mat_array)
fis_avg = np.mean(fis_array)
bio_avg = np.mean(bio_array)
qui_avg = np.mean(qui_array)
red_avg = np.mean(red_array)

In [34]:
# variance
mat_var = np.var(mat_array)
fis_var = np.var(fis_array)
bio_var = np.var(bio_array)
qui_var = np.var(qui_array)
red_var = np.var(red_array)

In [38]:
# create dictionary and dataframe

data_dict = {'Subject': ['Chemestry', 'Biology', 'Physics', 'Math', 'Essay'],
             'Number of Candidates': [qui_candidates, bio_candidats, fis_candidates, mat_candidates, red_candidates],
             'Average Score': [qui_avg, bio_avg, fis_avg, mat_avg, red_avg],
             'Variance Between Scores': [qui_var, bio_var, fis_var, mat_var, red_var],
             '10% Best Scores': [top_10_qui, top_10_bio, top_10_fis, top_10_mat, top_10_red]}

analysis_df = pd.DataFrame(data_dict)

In [39]:
analysis_df

,Subject,Number of Candidates,Average Score,Variance Between Scores,10% Best Scores
0,Chemestry,14459,5.807646,17.980822,12.00
1,Biology,16152,5.418880,14.984787,11.25
2,Physics,3882,8.013266,31.888284,16.50
3,Math,6152,7.992360,23.871894,15.50
4,Essay,32173,11.221552,11.702072,16.00
